In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00


In [2]:
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 5.4 MB/s eta 0:00:00


In [3]:
# connect ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.2.8
setup Colab for PySpark 3.2.3 and Spark NLP 4.2.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.7 MB/s eta 0:00:00


In [5]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [6]:
from transformers import BertTokenizer, BertForTokenClassification
#from transformers import AutoTokenizer, AutoModelForTokenClassification, TFAutoModelForTokenClassification

In [7]:
from sparknlp.annotator import *

MODEL_NAME = '/content/drive/MyDrive/Data_Science/thesis/ML_NER/NERModel_config'
bert = BertForTokenClassification.loadSavedModel(
     '{}/converting/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

In [9]:
bert.write().overwrite().save("./{}".format(MODEL_NAME))

In [10]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

In [11]:
tokenClassifier_loaded.getClasses()

['I-TOOL',
 'B-TOOL',
 'I-KNOW',
 '[SEP]',
 'B-LANG',
 'I-LANG',
 'B-FRAM',
 'I-FRAM',
 'B-KNOW',
 'I-PLAT',
 '[CLS]',
 'O',
 'B-PLAT']

In [28]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded    
])

In [47]:
df = spark.read.option('header','true').csv('/content/drive/MyDrive/Data_Science/thesis/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)

In [48]:
from pyspark.sql.functions import lower, col
df = df.select(df.SkillWillLearn.alias('description'))
df = df.withColumn('description', lower(col('description')))

df.show()

+--------------------+
|         description|
+--------------------+
|machine learning ...|
|"in an age now dr...|
|in this course, y...|
|explain the princ...|
|in the first cour...|
|define data scien...|
|this mooc – a joi...|
|describe the use ...|
|while telling sto...|
|in this course, y...|
|properly identify...|
|development of an...|
|describe differen...|
|distinguish betwe...|
|apply tidyverse f...|
|describe differen...|
|articulate differ...|
|build an investme...|
|in this course yo...|
|the capstone proj...|
+--------------------+
only showing top 20 rows



In [49]:
# couple of simple examples
#example = spark.createDataFrame([["Learn and Master software testing Quickly from the experts - GUARANTEED! THE IN-DEPTH software testing TRAINING - By SoftwareTestingHelp Team. on Udemy in the software testing category!"], ['My name is Clara and I live in Berkeley, California.']]).toDF("text")

result = pipeline.fit(df).transform(df)

# result is a DataFrame
result.select("description", "ner.result").show()



+--------------------+--------------------+
|         description|              result|
+--------------------+--------------------+
|machine learning ...|[B-KNOW, I-KNOW, ...|
|"in an age now dr...|[O, O, O, O, O, O...|
|in this course, y...|[O, O, O, O, O, O...|
|explain the princ...|[O, O, O, O, B-KN...|
|in the first cour...|[O, O, O, O, O, O...|
|define data scien...|[O, B-KNOW, I-KNO...|
|this mooc – a joi...|[O, O, O, O, O, O...|
|describe the use ...|[O, O, O, O, B-KN...|
|while telling sto...|[O, B-KNOW, I-KNO...|
|in this course, y...|[O, O, O, O, O, O...|
|properly identify...|[O, O, O, O, O, O...|
|development of an...|[O, O, O, O, O, O...|
|describe differen...|[O, O, O, O, O, O...|
|distinguish betwe...|[O, O, O, O, O, O...|
|apply tidyverse f...|[O, B-FRAM, O, O,...|
|describe differen...|[O, O, O, O, B-KN...|
|articulate differ...|[O, O, O, O, O, O...|
|build an investme...|[O, O, O, O, O, O...|
|in this course yo...|[O, O, O, O, O, O...|
|the capstone proj...|[O, O, O, 

In [53]:
result = result.select("description", "result")
result = result.withColumn("result", col("result").cast("string"))

In [35]:
result.show()

+--------------------+--------------------+--------------------+--------------------+
|         description|            document|               token|                 ner|
+--------------------+--------------------+--------------------+--------------------+
|Machine learning ...|[{document, 0, 17...|[{token, 0, 6, Ma...|[{named_entity, 0...|
|"In an age now dr...|[{document, 0, 36...|[{token, 0, 0, ",...|[{named_entity, 0...|
|In this course, y...|[{document, 0, 93...|[{token, 0, 1, In...|[{named_entity, 0...|
|Explain the princ...|[{document, 0, 25...|[{token, 0, 6, Ex...|[{named_entity, 0...|
|In the first cour...|[{document, 0, 88...|[{token, 0, 1, In...|[{named_entity, 0...|
|Define data scien...|[{document, 0, 32...|[{token, 0, 5, De...|[{named_entity, 0...|
|This MOOC – a joi...|[{document, 0, 19...|[{token, 0, 3, Th...|[{named_entity, 0...|
|Describe the use ...|[{document, 0, 33...|[{token, 0, 7, De...|[{named_entity, 0...|
|While telling sto...|[{document, 0, 10...|[{token, 0,

In [42]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

result = result.withColumn("token", col("token").cast("string")).withColumn("ner", col("ner").cast("string")).withColumn("document", col("document").cast("string"))



In [43]:
result.show()

+--------------------+--------------------+--------------------+--------------------+
|         description|            document|               token|                 ner|
+--------------------+--------------------+--------------------+--------------------+
|Machine learning ...|[{document, 0, 17...|[{token, 0, 6, Ma...|[{named_entity, 0...|
|"In an age now dr...|[{document, 0, 36...|[{token, 0, 0, ",...|[{named_entity, 0...|
|In this course, y...|[{document, 0, 93...|[{token, 0, 1, In...|[{named_entity, 0...|
|Explain the princ...|[{document, 0, 25...|[{token, 0, 6, Ex...|[{named_entity, 0...|
|In the first cour...|[{document, 0, 88...|[{token, 0, 1, In...|[{named_entity, 0...|
|Define data scien...|[{document, 0, 32...|[{token, 0, 5, De...|[{named_entity, 0...|
|This MOOC – a joi...|[{document, 0, 19...|[{token, 0, 3, Th...|[{named_entity, 0...|
|Describe the use ...|[{document, 0, 33...|[{token, 0, 7, De...|[{named_entity, 0...|
|While telling sto...|[{document, 0, 10...|[{token, 0,

In [54]:
result.write.options(header='True', delimiter=',') \
 .csv("/content/drive/MyDrive/Data_Science/thesis/ML_NER/result")